In [1]:
from vllm import LLM, RequestOutput, SamplingParams

2024-11-04 03:44:05.175837: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-04 03:44:05.266302: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8463] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-04 03:44:05.293321: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
LLM_params = {
    "model": "deepseek-ai/DeepSeek-Prover-V1.5-RL",
    "max_num_batched_tokens": 8192,
    "trust_remote_code": True,
}
llm = LLM(**LLM_params)

INFO 11-04 03:44:25 llm_engine.py:226] Initializing an LLM engine (v0.6.1.dev238+ge2c6e0a82) with config: model='deepseek-ai/DeepSeek-Prover-V1.5-RL', speculative_config=None, tokenizer='deepseek-ai/DeepSeek-Prover-V1.5-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=deepseek-ai/DeepSeek-Prover-V1.5-RL, use_v2_block_manager=False, num_scheduler_steps=

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

INFO 11-04 03:44:30 model_runner.py:1014] Starting to load model deepseek-ai/DeepSeek-Prover-V1.5-RL...
INFO 11-04 03:44:30 weight_utils.py:242] Using model weights format ['*.safetensors']


model-00002-of-000002.safetensors:  44%|####4     | 2.32G/5.23G [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-04 03:45:44 model_runner.py:1025] Loading model weights took 12.8725 GB
INFO 11-04 03:45:45 gpu_executor.py:122] # GPU blocks: 2893, # CPU blocks: 546
INFO 11-04 03:45:52 model_runner.py:1329] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-04 03:45:52 model_runner.py:1333] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-04 03:46:09 model_runner.py:1456] Graph capturing finished in 18 secs.


In [11]:
sampling_params = SamplingParams(**{
    "max_tokens": 1024,
    "temperature": 0.0,
    "top_k": 1,
    "top_p": 1.0,
    "n": 1,
    "logprobs": True,
})
input_data = 'Complete the following Lean 4 code with explanatory comments.\n```lean4\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 50000\nset_option linter.all false\nopen BigOperators Real Nat Topology Rat\n\n\n/-- For what real value of $k$ is $\\frac{13-\\sqrt{131}}{4}$ a root of $2x^2-13x+k$? Show that it is $\\frac{19}{4}$.-/\ntheorem mathd_algebra_116 (k x : ℝ) (h₀ : x = (13 - Real.sqrt 131) / 4)\n    (h₁ : 2 * x ^ 2 - 13 * x + k = 0) : k = 19 / 4 := by\n\n'
print(input_data)
out = llm.generate(
    prompts=input_data,
    sampling_params=sampling_params,
    use_tqdm=True,
)

Complete the following Lean 4 code with explanatory comments.
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 50000
set_option linter.all false
open BigOperators Real Nat Topology Rat


/-- For what real value of $k$ is $\frac{13-\sqrt{131}}{4}$ a root of $2x^2-13x+k$? Show that it is $\frac{19}{4}$.-/
theorem mathd_algebra_116 (k x : ℝ) (h₀ : x = (13 - Real.sqrt 131) / 4)
    (h₁ : 2 * x ^ 2 - 13 * x + k = 0) : k = 19 / 4 := by




Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.73s/it, est. speed input: 41.05 toks/s, output: 77.23 toks/s]


In [17]:
print(out[0].__dict__.keys())

dict_keys(['request_id', 'prompt', 'prompt_token_ids', 'prompt_logprobs', 'outputs', 'finished', 'metrics', 'lora_request', 'encoder_prompt', 'encoder_prompt_token_ids'])


In [22]:
print(out[0].outputs[0].text)

  /-
  Given \( x = \frac{13 - \sqrt{131}}{4} \) and \( 2x^2 - 13x + k = 0 \), we need to find the value of \( k \).
  1. Substitute \( x = \frac{13 - \sqrt{131}}{4} \) into the equation \( 2x^2 - 13x + k = 0 \).
  2. Simplify the equation by clearing the fraction and expanding the terms.
  3. Use the identity \( (\sqrt{131})^2 = 131 \) to simplify the equation further.
  4. Solve for \( k \) by isolating it on one side of the equation.
  -/
  -- Substitute the given value of x into the equation.
  rw [h₀] at h₁
  -- Clear the fraction by multiplying through by 4.
  field_simp
  -- Expand and simplify the equation.
  ring_nf at h₁
  /- The equation now is:
     -169 / 8 + √131 ^ 2 * (1 / 8) + k = 0
  -/
  -- Simplify further using the identity (√131)^2 = 131.
  field_simp at h₁
  -- Normalize the numerical values.
  norm_num at h₁
  -- Rearrange the equation to solve for k.
  rw [eq_comm]
  -- Use linear arithmetic to solve for k.
  linarith
```


In [27]:
out[0].outputs[0].__dict__.keys()

dict_keys(['index', 'text', 'token_ids', 'cumulative_logprob', 'logprobs', 'finish_reason', 'stop_reason', 'lora_request'])

In [29]:
out[0].outputs[0].logprobs[:10]

[{207: Logprob(logprob=-0.006903250236064196, rank=1, decoded_token=' ')},
 {893: Logprob(logprob=-0.048611968755722046, rank=1, decoded_token=' /')},
 {12: Logprob(logprob=-1.5497195136049413e-06, rank=1, decoded_token='-')},
 {185: Logprob(logprob=-4.768370445162873e-07, rank=1, decoded_token='\n')},
 {207: Logprob(logprob=0.0, rank=1, decoded_token=' ')},
 {16199: Logprob(logprob=-0.0067267161794006824, rank=1, decoded_token=' Given')},
 {12837: Logprob(logprob=-0.3066086769104004, rank=1, decoded_token=' \\(')},
 {1376: Logprob(logprob=-1.1324817933200393e-05, rank=1, decoded_token=' x')},
 {403: Logprob(logprob=0.0, rank=1, decoded_token=' =')},
 {357: Logprob(logprob=0.0, rank=1, decoded_token=' \\')}]

In [59]:
def truncate(snippet: str) -> str:
    new_code = snippet
    if new_code.endswith('```'):
        new_code = new_code[:-3]

    lines = new_code.split('\n')
    def start_of_comment(line: str) -> bool:
        # get the first character that's not a space
        strip_str = line.lstrip()
        first_non_space = strip_str[0] if strip_str else ''
        return first_non_space in ['/', '-']
    for i in range(1, len(lines)):
        if start_of_comment(lines[i]) and not start_of_comment(lines[i-1]):
            new_code = '\n'.join(lines[:i]) + '\n'
            break
    return new_code

Beam search

In [35]:
import os

# Set the environment variable
os.environ["VLLM_ALLOW_DEPRECATED_BEAM_SEARCH"] = "1"

# Verify that it's set correctly (optional)
print("VLLM_ALLOW_DEPRECATED_BEAM_SEARCH:", os.getenv("VLLM_ALLOW_DEPRECATED_BEAM_SEARCH"))

VLLM_ALLOW_DEPRECATED_BEAM_SEARCH: 1


In [55]:
sampling_params = SamplingParams(**{
    "max_tokens": 100,
    "temperature": 0.0,
    "top_k": -1,
    "top_p": 1.0,
    "n": 5,
    "logprobs": True,
    "use_beam_search": True
})
input_data = 'Complete the following Lean 4 code with explanatory comments.\n```lean4\nimport Mathlib\nimport Aesop\n\nset_option maxHeartbeats 50000\nset_option linter.all false\nopen BigOperators Real Nat Topology Rat\n\n\n/-- What is the sum of the four positive factors of the positive integer value of $\\sqrt{196}$? Show that it is 24.-/\ntheorem mathd_numbertheory_35 (S : Finset ℕ) (h₀ : ∀ n : ℕ, n ∣ Nat.sqrt 196) :\n    (∑ k in S, k) = 24 := by\n\n\n  /-- Tactic state:\n  S : Finset ℕ\n  h₀ : ∀ (n : ℕ), n ∣ Nat.sqrt 196\n  ⊢ ∑ k ∈ S, k = 24\n  -/\n\n  --'
print(input_data)
out = llm.generate(
    prompts=input_data,
    sampling_params=sampling_params,
    use_tqdm=True,
)

Complete the following Lean 4 code with explanatory comments.
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 50000
set_option linter.all false
open BigOperators Real Nat Topology Rat


/-- What is the sum of the four positive factors of the positive integer value of $\sqrt{196}$? Show that it is 24.-/
theorem mathd_numbertheory_35 (S : Finset ℕ) (h₀ : ∀ n : ℕ, n ∣ Nat.sqrt 196) :
    (∑ k in S, k) = 24 := by


  /-- Tactic state:
  S : Finset ℕ
  h₀ : ∀ (n : ℕ), n ∣ Nat.sqrt 196
  ⊢ ∑ k ∈ S, k = 24
  -/

  --


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  2.00s/it, est. speed input: 115.28 toks/s, output: 250.60 toks/s]


In [56]:
print(input_data)

Complete the following Lean 4 code with explanatory comments.
```lean4
import Mathlib
import Aesop

set_option maxHeartbeats 50000
set_option linter.all false
open BigOperators Real Nat Topology Rat


/-- What is the sum of the four positive factors of the positive integer value of $\sqrt{196}$? Show that it is 24.-/
theorem mathd_numbertheory_35 (S : Finset ℕ) (h₀ : ∀ n : ℕ, n ∣ Nat.sqrt 196) :
    (∑ k in S, k) = 24 := by


  /-- Tactic state:
  S : Finset ℕ
  h₀ : ∀ (n : ℕ), n ∣ Nat.sqrt 196
  ⊢ ∑ k ∈ S, k = 24
  -/

  --


In [57]:
for i in range(5):
    print(out[0].outputs[i].text)
    print('-' * 80)

 We know that the square root of 196 is 14.
  have h₁ : Nat.sqrt 196 = 14 := by norm_num
  -- Rewrite the goal using the fact that the square root of 196 is 14.
  rw [h₁] at h₀
  -- The positive factors of 14 are 1, 2, 7, and 14.

--------------------------------------------------------------------------------
 We know that the square root of 196 is 14.
  have h₁ : Nat.sqrt 196 = 14 := by norm_num
  -- Rewrite the goal using the fact that the square root of 196 is 14.
  rw [h₁] at h₀ ⊢
  -- The positive factors of 14 are 1, 2, 7, and 
--------------------------------------------------------------------------------
 We need to show that the sum of the factors of the square root of 196 is 24.
  -- First, we calculate the square root of 196, which is 14.
  -- The positive factors of 14 are 1, 2, 7, and 14.
  -- We sum these factors: 1 + 2 + 7 + 14 = 24.
  simpa
--------------------------------------------------------------------------------
 We need to show that the sum of the factors of 

In [60]:
snippets = [truncate(out[0].outputs[i].text) for i in range(5)]

In [77]:
list(out[0].outputs[0].logprobs[:20][0].values())

[Logprob(logprob=-0.31686216592788696, rank=1, decoded_token=' We')]

In [79]:
# get logprobs of each snippet
# get the truncated prefix of each snippet and add the corresponding prefix of output logprobs
logprobs = []
for i in range(5):
    snippet = snippets[i]
    cum_logprob = 0
    prefix = ''
    for completion in out[0].outputs[i].logprobs:
        x = list(completion.values())[0]
        prefix += x.decoded_token
        cum_logprob += x.logprob
        if prefix == snippet:
            break
    logprobs.append(cum_logprob)

print(snippets)
print(logprobs)

[' We know that the square root of 196 is 14.\n  have h₁ : Nat.sqrt 196 = 14 := by norm_num\n', ' We know that the square root of 196 is 14.\n  have h₁ : Nat.sqrt 196 = 14 := by norm_num\n', ' We need to show that the sum of the factors of the square root of 196 is 24.\n', ' We need to show that the sum of the factors of the square root of 196 is 24.\n', ' We know that the positive factors of the positive integer value of \\(\\sqrt{196}\\) are 1, 2, 4, 7, 14, and 28.\n  have h₁ : ∀ n : ℕ, n ∣ Nat.sqrt 196 := h₀\n']
[-3.200847765470698, -3.200847765470698, -2.3386666488338435, -2.3386666488338435, -4.80659925507922]
